In [ ]:
%%capture
!unzip /content/openstax_probing_dataset_english_v4.zip

In [ ]:
%%capture
!pip install openai

In [ ]:
!rm -r /content/batch_files

In [ ]:
import hashlib

# hash function to give each unique prompt an ID
def md5_hash(string):
    string_bytes = string.encode('utf-8')
    md5_hash = hashlib.md5()
    md5_hash.update(string_bytes)
    md5_hash_hex = md5_hash.hexdigest()
    return md5_hash_hex

In [ ]:
!rm /content/batch_files/*

In [ ]:
len(id_to_lab.keys())

113307

In [ ]:
import pickle

filename = 'malamute_id_to_lab.pkl'

with open(filename, 'wb') as file:
    pickle.dump(id_to_lab, file)

print(f"Dictionary saved to {filename}")

Dictionary saved to malamute_id_to_lab.pkl


In [ ]:
with open(filename, 'rb') as file:
    test = pickle.load(file)

In [ ]:
len(test.keys())

113307

In [ ]:
!rm /content/batch_files/*

In [ ]:
# this code block creates batch job files for OpenAI's batch API

import os
import json
from tqdm import tqdm

directory = '/content/content/English_v5'

id_to_json_line = {}

unique_ids = set()


for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.json'):
        file_path = os.path.join(directory, filename)

        with open(file_path, 'r') as file:
            data = json.load(file)

            for k in data.keys():
              for level in data[k].keys():
                for prompt in data[k][level]:
                  unique_id = md5_hash(prompt['prompt'] + prompt['label'])

                  if unique_id not in unique_ids:
                    unique_ids.add(unique_id)

                    id_to_json_line[unique_id] = {"custom_id": "gpt-4o_" + unique_id,
                            "method": "POST",
                            "url": "/v1/chat/completions",
                            "body": {"model": "gpt-4o-2024-05-13",
                                    "messages": [{"role": "system",
                                                  "content": "You are a helpful, respectful and honest assistant. Your answers should be crisp, short and not repetitive. Here are 5 examples: Example 1: Prompt: Francesco Bartolomeo Conti was born in [MASK]. Answer: Florence. Example 2: Prompt: English bulldog is a subclass of [MASK]. Answer: dog. Example 3: Prompt: The official language of Mauritius is [MASK]. Answer: English. Example 4: Prompt: Nicotine binds to [MASK]. Answer: CHRNA4. Example 5: Prompt: Hepatitis has symptoms such as [MASK]. Answer: Abdominal pain."},
                                                    {"role": "user", "content": f"Prompt: {prompt['prompt']}"}],"max_tokens": 1000}}
                  else:
                    not_unique_count += 1

print(not_unique_count)



100%|██████████| 56/56 [00:02<00:00, 23.54it/s]

117981


In [ ]:
import os
import json
from tqdm import tqdm

directory = '/content/content/English_v5'


unique_ids = set(id_to_pred.keys())
not_unique_count = 0


id_to_lab = {}

for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.json'):
        file_path = os.path.join(directory, filename)

        with open(file_path, 'r') as file:
            data = json.load(file)

            output_file = "/content/batch_files/" + filename.split('.')[0] + '_openai_batch.jsonl'

            with open(output_file, 'a') as file:

              for k in data.keys():
                for level in data[k].keys():
                  for prompt in data[k][level]:
                    unique_id = md5_hash(prompt['prompt'] + prompt['label'])

                    if unique_id not in unique_ids:
                      unique_ids.add(unique_id)
                      id_to_lab[unique_id] = prompt['label']
                      line = {"custom_id": "gpt-4o_" + unique_id,
                              "method": "POST",
                              "url": "/v1/chat/completions",
                              "body": {"model": "gpt-4o-2024-05-13",
                                      "messages": [{"role": "system",
                                                    "content": "You are a helpful, respectful and honest assistant. Your answers should be crisp, short and not repetitive. Here are 5 examples: Example 1: Prompt: Francesco Bartolomeo Conti was born in [MASK]. Answer: Florence. Example 2: Prompt: English bulldog is a subclass of [MASK]. Answer: dog. Example 3: Prompt: The official language of Mauritius is [MASK]. Answer: English. Example 4: Prompt: Nicotine binds to [MASK]. Answer: CHRNA4. Example 5: Prompt: Hepatitis has symptoms such as [MASK]. Answer: Abdominal pain."},
                                                      {"role": "user", "content": f"Prompt: {prompt['prompt']}"}],"max_tokens": 1000}}
                      file.write(json.dumps(line) + '\n')
                    else:
                      not_unique_count += 1

print(not_unique_count)



100%|██████████| 56/56 [00:03<00:00, 15.65it/s]

1558


In [ ]:
file_path

'/content/batch_files/principles-microeconomics-3e_openai_batch.jsonl'

In [ ]:
filenames = []

for filename in tqdm(os.listdir('/content/batch_files')):
  filenames.append(filename)

100%|██████████| 56/56 [00:00<00:00, 245691.45it/s]


In [ ]:
queued_or_completed_book_ids = [x + '.jsonl' for x in unqueued_job_ids]

In [ ]:
len(set([filename for filename in filenames if filename not in queued_or_completed_book_ids]))

1

In [ ]:
unqueued_book_ids = [filename for filename in filenames if filename not in queued_or_completed_book_ids]

In [ ]:
len(set(processed_books))

33

In [ ]:
len(set([filename for filename in filenames if filename not in processed_books]))

23

In [ ]:
print(len([filename for filename in filenames if filename not in processed_books]))

23


In [ ]:
file_path

'/content/batch_files/principles-microeconomics-3e_openai_batch.jsonl'

In [ ]:
counter = 0
for f in os.listdir('/content/batch_files'):
  if f in batches_that_have_run:
    counter += 1

print(counter)

0


In [ ]:
set(os.listdir('/content/batch_files')) - set(batches_that_have_run)

{'.ipynb_checkpoints',
 'anatomy-and-physiology-2e_openai_batch.jsonl',
 'introduction-political-science_openai_batch.jsonl',
 'microbiology_openai_batch.jsonl',
 'world-history-volume-1_openai_batch.jsonl',
 'world-history-volume-2_openai_batch.jsonl'}

In [ ]:
import os

# utility to split up files that were too big
def split_jsonl(file_path, output_dir, num_parts=2):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    total_lines = len(lines)
    lines_per_part = total_lines // num_parts

    os.makedirs(output_dir, exist_ok=True)

    for i in range(num_parts):
        start = i * lines_per_part
        end = (i + 1) * lines_per_part if i < num_parts - 1 else total_lines
        part_lines = lines[start:end]

        part_file_path = os.path.join(output_dir, f"{os.path.basename(file_path).replace('.jsonl', f'_part_{i+1}.jsonl')}")
        with open(part_file_path, 'w') as part_file:
            part_file.writelines(part_lines)

    print(f"File '{file_path}' split into {num_parts} parts.")

file_names = [
    '/content/batch_files/anatomy-and-physiology-2e_openai_batch.jsonl',
    '/content/batch_files/introduction-political-science_openai_batch.jsonl',
    '/content/batch_files/microbiology_openai_batch.jsonl',
    '/content/batch_files/world-history-volume-1_openai_batch.jsonl',
    '/content/batch_files/world-history-volume-2_openai_batch.jsonl'
]

output_directory = 'split_files'

for file_name in file_names:
    split_jsonl(file_name, output_directory)


File '/content/batch_files/anatomy-and-physiology-2e_openai_batch.jsonl' split into 2 parts.
File '/content/batch_files/introduction-political-science_openai_batch.jsonl' split into 2 parts.
File '/content/batch_files/microbiology_openai_batch.jsonl' split into 2 parts.
File '/content/batch_files/world-history-volume-1_openai_batch.jsonl' split into 2 parts.
File '/content/batch_files/world-history-volume-2_openai_batch.jsonl' split into 2 parts.


In [ ]:
batch_input_file = client.files.create(
  file=open('/content/batch_files/nutrition_openai_batch.jsonl', "rb"),
  purpose="batch"
)

batch_input_file_id = batch_input_file.id


client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "gpt-4o_" + filename.split('.')[0]
    }
)

Batch(id='batch_9MXdy7s7fGE62Yt3Bs7Wh797', completion_window='24h', created_at=1722038690, endpoint='/v1/chat/completions', input_file_id='file-L4OMqlApfFSmyOHtbIX6lDLx', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1722125090, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'gpt-4o_college-physics-2e_openai_batch'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [ ]:
len(set(os.listdir('/content/batch_files')) - set(batches_that_have_run))

6

In [ ]:
split_batches_that_have_run = ['anatomy-and-physiology-2e_openai_batch_part_2.jsonl']

In [ ]:
set(os.listdir('/content/split_files')) - set(split_batches_that_have_run)

set()

In [ ]:
split_batches_that_have_run

['anatomy-and-physiology-2e_openai_batch_part_2.jsonl',
 'microbiology_openai_batch_part_1.jsonl',
 'introduction-political-science_openai_batch_part_2.jsonl',
 'anatomy-and-physiology-2e_openai_batch_part_1.jsonl',
 'introduction-political-science_openai_batch_part_1.jsonl',
 'world-history-volume-1_openai_batch_part_2.jsonl',
 'world-history-volume-2_openai_batch_part_1.jsonl',
 'world-history-volume-1_openai_batch_part_1.jsonl',
 'microbiology_openai_batch_part_2.jsonl',
 'world-history-volume-2_openai_batch_part_2.jsonl']

In [ ]:
# queues batch jobs

from openai import OpenAI
client = OpenAI()

for filename in tqdm(set(os.listdir('/content/split_files')) - set(split_batches_that_have_run)):
    if filename.endswith('.jsonl'):
        file_path = os.path.join('/content/split_files', filename)

        batch_input_file = client.files.create(
          file=open(file_path, "rb"),
          purpose="batch"
        )

        batch_input_file_id = batch_input_file.id


        client.batches.create(
            input_file_id=batch_input_file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h",
            metadata={

              "description": "gpt-4o_" + filename.split('.')[0]
            }
        )
        split_batches_that_have_run.append(filename)
        break

  0%|          | 0/1 [00:01<?, ?it/s]


In [ ]:
# queues batch jobs

from openai import OpenAI
client = OpenAI()

for filename in tqdm(set(os.listdir('/content/batch_files')) - set(batches_that_have_run)):
    if filename.endswith('.jsonl'):
        file_path = os.path.join('/content/batch_files', filename)

        batch_input_file = client.files.create(
          file=open(file_path, "rb"),
          purpose="batch"
        )

        batch_input_file_id = batch_input_file.id


        client.batches.create(
            input_file_id=batch_input_file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h",
            metadata={

              "description": "gpt-4o_" + filename.split('.')[0]
            }
        )

100%|██████████| 6/6 [00:07<00:00,  1.20s/it]


In [ ]:
# checks status of jobs

from collections import defaultdict

data_dicts = []

unqueued_job_ids = []

status_counts = defaultdict(int)

for data in client.batches.list():
  data_dict = data.to_dict()
  if data_dict['status'] not in ['failed', 'cancelled', 'cancelling']:
    status_counts[data_dict['status']] += 1
    data_dicts.append(data_dict)

    unqueued_job_ids.append(data_dict['metadata']['description'])

print(status_counts)

defaultdict(<class 'int'>, {'finalizing': 1, 'completed': 156})


In [ ]:
len(unqueued_job_ids)

65

In [ ]:
data_dicts = []

for data in client.batches.list(limit=100):
  data_dicts.append(data.to_dict())

In [ ]:
gpt4o_dicts = [dct for dct in data_dicts if 'metadata' in dct.keys() and 'gpt-4o' in dct['metadata']['description']]

In [ ]:
len(gpt4o_dicts)

54

In [ ]:
data_dicts = []
statuses = []
metadatum = []

for data in client.batches.list():
  data_dict = data.to_dict()

  if data_dict['status'] not in ['failed', 'cancelled', 'cancelling']:
    data_dicts.append(data_dict)
    statuses.append(data_dict['status'])
    metadatum.append(data_dict['metadata'])

In [ ]:
batches_that_have_run = list(set([m['description'][7:]+'.jsonl' for m in metadatum if "gpt-4o" in m['description']]))

In [ ]:
len(batches_that_have_run)

58

In [ ]:
processed_books = [x['description'] + '.jsonl' for x in metadatum]

In [ ]:
processed_books

['statistics_openai_batch.jsonl',
 'principles-macroeconomics-3e_openai_batch.jsonl',
 'college-success_openai_batch.jsonl',
 'college-algebra-2e_openai_batch.jsonl',
 'introductory-business-statistics-2e_openai_batch.jsonl',
 'chemistry-atoms-first-2e_openai_batch.jsonl',
 'organizational-behavior_openai_batch.jsonl',
 'introduction-python-programming_openai_batch.jsonl',
 'concepts-biology_openai_batch.jsonl',
 'preparing-for-college-success_openai_batch.jsonl',
 'introduction-intellectual-property_openai_batch.jsonl',
 'microbiology_openai_batch.jsonl',
 'university-physics-volume-2_openai_batch.jsonl',
 'contemporary-mathematics_openai_batch.jsonl',
 'college-success-concise_openai_batch.jsonl',
 'chemistry-2e_openai_batch.jsonl',
 'anatomy-and-physiology-2e_openai_batch.jsonl',
 'introduction-political-science_openai_batch.jsonl',
 'introduction-philosophy_openai_batch.jsonl',
 'university-physics-volume-3_openai_batch.jsonl',
 'psychology-2e_openai_batch.jsonl',
 'principles-mark

In [ ]:
unprocessed_books = []

for filename in tqdm(os.listdir('/content/batch_files')):
    if filename.endswith('.jsonl') and filename not in processed_books:
        file_path = os.path.join('/content/batch_files', filename)

        unprocessed_books.append(filename)

100%|██████████| 56/56 [00:00<00:00, 129625.29it/s]


In [ ]:
len(unprocessed_books)

23

In [ ]:
processed_books

['statistics_openai_batch.jsonl',
 'principles-macroeconomics-3e_openai_batch.jsonl',
 'college-success_openai_batch.jsonl',
 'college-algebra-2e_openai_batch.jsonl',
 'introductory-business-statistics-2e_openai_batch.jsonl',
 'chemistry-atoms-first-2e_openai_batch.jsonl',
 'organizational-behavior_openai_batch.jsonl',
 'introduction-python-programming_openai_batch.jsonl',
 'concepts-biology_openai_batch.jsonl',
 'preparing-for-college-success_openai_batch.jsonl',
 'introduction-intellectual-property_openai_batch.jsonl',
 'microbiology_openai_batch.jsonl',
 'university-physics-volume-2_openai_batch.jsonl',
 'contemporary-mathematics_openai_batch.jsonl',
 'college-success-concise_openai_batch.jsonl',
 'chemistry-2e_openai_batch.jsonl',
 'anatomy-and-physiology-2e_openai_batch.jsonl',
 'introduction-political-science_openai_batch.jsonl',
 'introduction-philosophy_openai_batch.jsonl',
 'university-physics-volume-3_openai_batch.jsonl',
 'psychology-2e_openai_batch.jsonl',
 'principles-mark

In [ ]:
unprocessed_books

['writing-guide_openai_batch.jsonl',
 'introduction-business_openai_batch.jsonl',
 'calculus-volume-2_openai_batch.jsonl',
 'principles-management_openai_batch.jsonl',
 'principles-finance_openai_batch.jsonl',
 'college-physics-2e_openai_batch.jsonl',
 'university-physics-volume-1_openai_batch.jsonl',
 'astronomy-2e_openai_batch.jsonl',
 'intermediate-algebra-2e_openai_batch.jsonl',
 'principles-financial-accounting_openai_batch.jsonl',
 'nutrition_openai_batch.jsonl',
 'world-history-volume-2_openai_batch.jsonl',
 'algebra-and-trigonometry-2e_openai_batch.jsonl',
 'world-history-volume-1_openai_batch.jsonl',
 'precalculus-2e_openai_batch.jsonl',
 'introduction-anthropology_openai_batch.jsonl',
 'calculus-volume-1_openai_batch.jsonl',
 'american-government-3e_openai_batch.jsonl',
 'prealgebra-2e_openai_batch.jsonl',
 'introductory-statistics-2e_openai_batch.jsonl',
 'principles-microeconomics-3e_openai_batch.jsonl',
 'elementary-algebra-2e_openai_batch.jsonl',
 'workplace-software-skil

In [ ]:
len(processed_books)

43

In [ ]:
unqueued_book_ids

['principles-microeconomics-3e_openai_batch.jsonl',
 'elementary-algebra-2e_openai_batch.jsonl',
 'workplace-software-skills_openai_batch.jsonl']

In [ ]:
file_path

'/content/batch_files/principles-microeconomics-3e_openai_batch.jsonl'

In [ ]:
for filename in tqdm(os.listdir('/content/batch_files')):
    if filename.endswith('.jsonl') and filename in unqueued_book_ids:
        file_path = os.path.join('/content/batch_files', filename)

        try:
          batch_input_file = client.files.create(
            file=open(file_path, "rb"),
            purpose="batch"
          )

          batch_input_file_id = batch_input_file.id

          client.batches.create(
              input_file_id=batch_input_file_id,
              endpoint="/v1/chat/completions",
              completion_window="24h",
              metadata={
                "description": filename.split('.')[0]
              }
          )
        except:
          pass

100%|██████████| 56/56 [00:03<00:00, 16.29it/s] 


In [ ]:
for data in client.batches.list(limit=100):
  data_dict = data.to_dict()

  file_id = data_dict['output_file_id']

  if file_id is not None:
    file_response = client.files.content(file_id)
    print(file_response.text)
    break

{"id": "batch_req_8vlCYZRdPSCCGa11P0E1Wpun", "custom_id": "57ac6d9bd4a076c084ff1b35fe9b8d3c", "response": {"status_code": 200, "request_id": "4e6c4797fe692586dd32bf0774b0e0c9", "body": {"id": "chatcmpl-9p3LFUEc3FtxVwM137T03HSgfEi6P", "object": "chat.completion", "created": 1721954601, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "t-distribution."}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 192, "completion_tokens": 4, "total_tokens": 196}, "system_fingerprint": "fp_661538dc1f"}}, "error": null}
{"id": "batch_req_Bvsu5fby9M0f7z9bVJuSbT0W", "custom_id": "b8a76806502d93f6b04bc185ef489948", "response": {"status_code": 200, "request_id": "44829901cca93ff591df1bd80b8ea7d5", "body": {"id": "chatcmpl-9p3LKEvBrJGqB2FAr14H5IRLePAuk", "object": "chat.completion", "created": 1721954606, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "Answer: t-test."}, "

In [ ]:
file_response = client.files.content('file-7JmoKeEfrZ2BCbFXwYeUJZxo')
print(file_response.text)

{"custom_id": "cc0c50bc502145d975b85728c96cabdf", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o-mini-2024-07-18", "messages": [{"role": "system", "content": "You are a helpful, respectful and honest assistant. Your answers should be crisp, short and not repetitive. Here are 5 examples: Example 1: Prompt: Francesco Bartolomeo Conti was born in [MASK]. Answer: Florence. Example 2: Prompt: English bulldog is a subclass of [MASK]. Answer: dog. Example 3: Prompt: The official language of Mauritius is [MASK]. Answer: English. Example 4: Prompt: Nicotine binds to [MASK]. Answer: CHRNA4. Example 5: Prompt: Hepatitis has symptoms such as [MASK]. Answer: Abdominal pain."}, {"role": "user", "content": "Prompt: According to [MASK], low-income countries are affected by their lack of industrialization and can improve their global economic standing through (Armer and Katsillis 2010):"}], "max_tokens": 1000}}
{"custom_id": "a383d7d52b7bb84940ae5240d99892e0", "method": "POS

In [ ]:
len(data_dicts)

43

In [ ]:
client.batches.cancel("batch_abc123")

In [ ]:
batch_data = client.batches.list().data

In [ ]:
batch_data[0].id

'batch_5ydeNba4qj8yJhhnKzI5wRo7'

In [ ]:
batch_input_file = client.files.create(
          file=open(file_path, "rb"),
          purpose="batch"
        )

        batch_input_file_id = batch_input_file.id

        client.batches.create(
            input_file_id=batch_input_file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h",
            metadata={
              "description": filename.split('.')[0]
            }
        )

'/content/content/English_v5/biology-2e.json'

In [ ]:
from openai import OpenAI
client = OpenAI()

In [ ]:
texts =[]

for data in client.batches.list():
  data_dict = data.to_dict()
  if data_dict['status'] not in ['failed', 'cancelled', 'cancelling'] and 'gpt-4o' in data_dict['metadata']['description']:


    file_response = client.files.content(data_dict['output_file_id'])

    texts.append(file_response.text)


In [ ]:
len(texts)

88

In [ ]:
import json
from tqdm import tqdm

In [ ]:
id_to_pred = {}

for text in tqdm(texts):
  lines = text.split('\n')

  for line in lines[:-1]:
    x = json.loads(line)
    id_to_pred[x['custom_id']] = x['response']['body']['choices'][0]['message']['content']


100%|██████████| 88/88 [00:02<00:00, 36.37it/s]


In [ ]:
len(id_to_pred)

113307

In [ ]:
texts[0].split('\n')[0]

'{"id": "batch_req_whKrEaNM3bsRPAMECNOH8AqB", "custom_id": "5328ae1079cbed94efae2c8159b267a1", "response": {"status_code": 200, "request_id": "f51b869773cf0589b9393fb5dac60e72", "body": {"id": "chatcmpl-9pIhKuX36JIpk7YoutU9tw0K33HRT", "object": "chat.completion", "created": 1722013630, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "ribbon."}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 216, "completion_tokens": 3, "total_tokens": 219}, "system_fingerprint": "fp_611b667b19"}}, "error": null}'

In [ ]:
with open('malamute_id_to_pred.json', 'w') as json_file:
    json.dump(id_to_pred, json_file, indent=4)

print(f"Dictionary successfully dumped to {filename}")

Dictionary successfully dumped to workplace-software-skills_openai_batch.jsonl


In [ ]:
json.loads(texts[0])

JSONDecodeError: Extra data: line 2 column 1 (char 581)

In [ ]:
import os
import json
from tqdm import tqdm

directory = '/content/content/English_v5'


st_level_total = 0
st_level_correct = 0
pg_level_total = 0
pg_level_correct = 0

for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.json'):
        file_path = os.path.join(directory, filename)

        with open(file_path, 'r') as file:
            data = json.load(file)

            for k in data.keys():
              for level in data[k].keys():
                for prompt in data[k][level]:

                  if not prompt['ignore']
                  unique_id = md5_hash(prompt['prompt'] + prompt['label'])


                  if prompt['label'] in id_to_pred[unique_id]:
                    if level == 'paragraph_level':
                      pg_level_correct += 1
                    elif level == 'sentence_level':
                      st_level_correct += 1
                    else:
                      print("ERROR")
                  if level == 'paragraph_level':
                      pg_level_total += 1
                  elif level == 'sentence_level':
                      st_level_total += 1




100%|██████████| 56/56 [00:02<00:00, 24.66it/s]


In [ ]:
st_level_correct/st_level_total

0.42134953745692

In [ ]:
st_level_correct/st_level_total
pg_level_correct/pg_level_total

0.5234117351636394

In [ ]:
with open('/content/malamute_id_to_pred.json', 'rb') as f:
  mini_id_to_pred = json.load(f)

In [ ]:
len(set(id_to_json_line.keys()) - set([key[7:] for key in list(id_to_pred.keys())]))

12952

In [ ]:
sub_dict = {k:id_to_json_line[k] for k in list(set(id_to_json_line.keys()) - set([key[7:] for key in list(id_to_pred.keys())]))}

In [ ]:
values_list = list(sub_dict.values())

batch_size = len(values_list) // 4
remainder = len(values_list) % 4

batches = []
start_index = 0
for i in range(4):
    end_index = start_index + batch_size + (1 if i < remainder else 0)
    batches.append(values_list[start_index:end_index])
    start_index = end_index

for i, batch in enumerate(batches):
    file_name = f'/content/misc_batches/misc_batch_{i+1}.jsonl'
    with open(file_name, 'w') as f:
        for item in batch:
            f.write(json.dumps(item) + '\n')

Files created successfully.


In [ ]:
mixed_batches_that_have_run = []

In [ ]:
from openai import OpenAI
client = OpenAI()



# RUN BATCHES
for filename in tqdm(set(os.listdir('/content/misc_batches')) - set(mixed_batches_that_have_run)):
    if filename.endswith('.jsonl'):  # Check if the file is a JSON file
        file_path = os.path.join('/content/misc_batches', filename)

        batch_input_file = client.files.create(
          file=open(file_path, "rb"),
          purpose="batch"
        )

        batch_input_file_id = batch_input_file.id


        client.batches.create(
            input_file_id=batch_input_file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h",
            metadata={

              "description": "gpt-4o_" + filename.split('.')[0]
            }
        )
        mixed_batches_that_have_run.append(filename)
        break

  0%|          | 0/1 [00:01<?, ?it/s]


In [ ]:
list(id_to_pred.keys())[0]

'gpt-4o_b02e00f21e2090bab1a384f7345b198e'

In [ ]:
gpt4_id_to_pred = {k[7:]:id_to_pred[k] for k in id_to_pred.keys()}

In [ ]:
len(set(id_to_pred.keys()))

113307

In [ ]:
import os
import json
from tqdm import tqdm

directory = '/content/content/English_v5'


st_level_total = 0
st_level_correct = 0
pg_level_total = 0
pg_level_correct = 0

for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.json'):
        file_path = os.path.join(directory, filename)

        with open(file_path, 'r') as file:
            data = json.load(file)

            for k in data.keys():
              for level in data[k].keys():
                for prompt in data[k][level]:
                  unique_id = md5_hash(prompt['prompt'] + prompt['label'])


                  if prompt['label'] in gpt4_id_to_pred[unique_id]:
                    if level == 'paragraph_level':
                      pg_level_correct += 1
                    elif level == 'sentence_level':
                      st_level_correct += 1
                    else:
                      print("ERROR")
                  if level == 'paragraph_level':
                      pg_level_total += 1
                  elif level == 'sentence_level':
                      st_level_total += 1




100%|██████████| 56/56 [00:01<00:00, 33.83it/s]


In [ ]:
print(st_level_correct/st_level_total)

0.49580990386359514


In [ ]:
print(pg_level_correct/pg_level_total)

0.5631204486481962


In [ ]:
with open('malamute_id_to_pred_gpt-4o.json', 'w') as json_file:
    json.dump(gpt4_id_to_pred, json_file, indent=4)

Dictionary successfully dumped to astronomy-2e.json
